In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
import plotly.express as px
from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler
import plotly.graph_objects as go
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn.utils import class_weight
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split,KFold, GroupKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_squared_log_error

<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> NMF  Non-Negative Matrix Factorization </h1>

# This notebook is kind of spinnof of my [PCA from scratch](https://www.kaggle.com/marcinstasko/pca-analysis-tutorial-from-scratch). 

As we have tested many suprvised method in this competition without spectacular sucess I will test NMF with is an example of unsupervised feature selection. We will does not analyse the y labels. <br>
# If you like this notboook let me know. I will concider to make it as tutorial


<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> 2. Initial Setup-Data Preparaton </h1>


In [ ]:
# Initialize local variables

SEED = 1992
TARGET = ['target']
SAMPLE = 2000 #For Visualization purposees

In [ ]:
# Import the data and prepare for the analysis. I have used sampling for speed up purposes. You can remove it :)

train = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

#Prepare train and test dataset
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

#Checking the null Data
null_data = (train.isna().sum().sort_values(ascending=False) / len(train) * 100) 
fig, ax = plt.subplots(1,1,figsize=(35, 7)) 
ax.bar(null_data.index, 100, color='#dadada', width=0.6) 
bar = ax.bar(null_data.index,null_data, width=0.6) 
ax.bar_label(bar, fmt='%.01f %%') 
ax.spines.left.set_visible(False) 
ax.set_yticks([]) 
ax.set_title('Null Data Ratio', fontweight='bold') 
plt.show()

train.describe().drop('count').T\
        .style.bar(subset=['mean'])\
        .background_gradient(subset=['std'])\
        .background_gradient(subset=['50%'])\
        .background_gradient(subset=['max'])

<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> Dataset Preparation </h1>


In [ ]:
def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

all_df = pd.concat([train, test]).reset_index(drop=True)
all_df[TARGET]=label_encoder(all_df[TARGET])
train_last_id = train.shape[0]

X = all_df.drop(TARGET,axis=1)
y = all_df[TARGET]
print('Feature dataset format:\t{}\nTarget dataset format:\t{}\nTrain dataset size:\t{}'.format(X.shape,y.shape,train_last_id))

In [ ]:
# Calculate the classes weights
class_ratio = class_weight.compute_class_weight('balanced', y.target.unique(), y.target)
print(class_ratio)

In [ ]:
#Variable standarization and preparation
X_scaled = MinMaxScaler().fit_transform(X)
train_X_to_select = X_scaled[:train_last_id]
test_X_to_select = X_scaled[train_last_id:]

train_y_to_select = y[:train_last_id]
test_y_to_select = y[train_last_id:]

In [ ]:
# How scaled matrix looks like
X_scaled

<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;">NMF Applicaton</h1>


In [ ]:
num_classes = len(class_ratio)

#Initialize NMF instance
nmf = NMF(n_components=num_classes, init='random', random_state=SEED)

#Weights calculation
W_train = nmf.fit_transform(train_X_to_select)
W_test = nmf.fit_transform(test_X_to_select)

In [ ]:
# Create column labels accoring to nmf components number

column_labels = []
for i in range(num_classes):
        column_labels.append('NMFClass_{}'.format(i+1))

NMF_DF = pd.DataFrame(W_train , columns = column_labels)
NMF_DF = pd.concat([NMF_DF , y] , axis = 1)

<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> Prediction </h1>
Sample prediction 


In [ ]:
lgbm_params = {
    'n_estimators': 45000,
    'objective' : 'multiclass',
    'metric' : 'multi_logloss',
    'random_state': SEED,
    'learning_rate': 0.02,
    'min_child_samples': 150,
    'reg_alpha': 750,
    'reg_lambda': 9e-2,
    'num_leaves': 20,
    'max_depth': 3,#16
    'colsample_bytree': 0.15,
    'subsample': 0.7,
    'subsample_freq': 2,
    'max_bin': 240,
    'device':'gpu'
}

In [ ]:
cols = list(NMF_DF.columns)
cols.remove("target")

NMF_train = NMF_DF[:train_last_id]
NMF_test = NMF_DF[train_last_id:]

In [ ]:
def KFoldTraining(classifier):
    test_preds = None
    train_rmse = 0
    val_rmse = 0
    n_splits = 5

    classifier = classifier
    

    kf = KFold(n_splits = n_splits , shuffle = True , random_state = 42)
    for fold, (tr_index , val_index) in enumerate(kf.split(NMF_train[cols].values , NMF_train['target'].values)):

        print("-" * 50)
        print(f"Fold {fold + 1}")

        x_train,x_val = NMF_DF[cols].values[tr_index] , NMF_train[cols].values[val_index]
        y_train,y_val = NMF_DF['target'].values[tr_index] , NMF_train['target'].values[val_index]

        eval_set = [(x_val, y_val)]

        model = classifier
        model.fit(x_train, y_train, eval_set = eval_set, verbose = 1000)

        train_preds = model.predict(x_train)
        train_rmse += mean_squared_error(y_train ,train_preds , squared = False)
        print("Training RMSE : " , mean_squared_error(y_train ,train_preds , squared = False))

        val_preds = model.predict(x_val)
        val_rmse += mean_squared_error(y_val , val_preds , squared = False)
        print("Validation RMSE : " , mean_squared_error(y_val , val_preds , squared = False))

        if test_preds is None:
            test_preds = model.predict_proba(NMF_test[cols].values)
        else:
            test_preds += model.predict_proba(NMF_test[cols].values)

    print("-" * 200)
    print("Average Training RMSE : " , train_rmse / n_splits)
    print("Average Validation RMSE : " , val_rmse / n_splits)

    test_preds /= n_splits
    return test_preds

In [ ]:
LGBM_classifier = LGBMClassifier(**lgbm_params)
LGBM_pred = KFoldTraining(LGBM_classifier)


test_preds = LGBM_pred.copy() # Next I will make model ensembing

In [ ]:
sample_submission['Class_1']=test_preds[:,0]
sample_submission['Class_2']=test_preds[:,1]
sample_submission['Class_3']=test_preds[:,2]
sample_submission['Class_4']=test_preds[:,3]
sample_submission.head()

In [ ]:
sample_submission.to_csv("NMF_1.csv",index=False)

<h1 style="background-color:#A50034; font-family:segoeui; font-size:200%; text-align:center; border-radius: 15px 50px;"> Notebook development plan</h1>

# If you like the notebook I will be happy. This motivate me to develop it :) <br>
# Plans
- Making notebook as NMF tutorial - if you will be interested in
- The ideas in comments<br>

Lets connect on [LinkedIn](https://www.linkedin.com/in/marcinstasko/?locale=en_US) :)